In [ ]:
#Elo Rating and Win Percentage
#By: Anthony F.


import math

class EloRatingSystem:
    def __init__(self, k=32, initial_rating=1500):
        self.k = k  # Standard Elo adjustment factor
        self.ratings = {}  # Dictionary to store team ratings
        self.initial_rating = initial_rating  # Default initial Elo rating


    def expected_score(self, rating1, rating2):
        return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

    def update_ratings(self, team1, team2, score1, score2):
        if team1 not in self.ratings:
            self.ratings[team1] = self.initial_rating
        if team2 not in self.ratings:
            self.ratings[team2] = self.initial_rating

        rating1, rating2 = self.ratings[team1], self.ratings[team2]
        exp_score1 = self.expected_score(rating1, rating2)
        exp_score2 = self.expected_score(rating2, rating1)

        actual_score1 = 1 if score1 > score2 else 0.5 if score1 == score2 else 0
        actual_score2 = 1 - actual_score1

        # Margin of Victory Multiplier
        mov = abs(score1 - score2)
        mov_multiplier = math.log(mov + 1) * (2.2 / (1 + 0.001 * abs(rating1 - rating2)))

        # Update ratings
        self.ratings[team1] += self.k * mov_multiplier * (actual_score1 - exp_score1)
        self.ratings[team2] += self.k * mov_multiplier * (actual_score2 - exp_score2)

    def normalize_end_of_season(self):
        """Applies regression to the mean to avoid long-term inflation/deflation."""
        for team in self.ratings:
            if self.ratings[team] > self.initial_rating:
                self.ratings[team] -= (self.ratings[team] - self.initial_rating) * 0.25
            else:
                self.ratings[team] += (self.initial_rating - self.ratings[team]) * 0.25

    def get_ratings(self):
        return self.ratings







In [ ]:
from google.colab import files

# This will prompt you to upload files
uploaded = files.upload()

# Displaying the names of the uploaded files
for filename in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=filename, length=len(uploaded[filename])))




Saving ncaa_mens_lacrosse_results_2014.csv to ncaa_mens_lacrosse_results_2014 (1).csv
Saving ncaa_mens_lacrosse_results_2015.csv to ncaa_mens_lacrosse_results_2015 (1).csv
Saving ncaa_mens_lacrosse_results_2016.csv to ncaa_mens_lacrosse_results_2016 (1).csv
Saving ncaa_mens_lacrosse_results_2017.csv to ncaa_mens_lacrosse_results_2017 (1).csv
Saving ncaa_mens_lacrosse_results_2018.csv to ncaa_mens_lacrosse_results_2018 (1).csv
Saving ncaa_mens_lacrosse_results_2019.csv to ncaa_mens_lacrosse_results_2019 (1).csv
Saving ncaa_mens_lacrosse_results_2020.csv to ncaa_mens_lacrosse_results_2020 (1).csv
Saving ncaa_mens_lacrosse_results_2021.csv to ncaa_mens_lacrosse_results_2021 (1).csv
Saving ncaa_mens_lacrosse_results_2022.csv to ncaa_mens_lacrosse_results_2022 (1).csv
Saving ncaa_mens_lacrosse_results_2023.csv to ncaa_mens_lacrosse_results_2023 (1).csv
Saving ncaa_mens_lacrosse_results_2024.csv to ncaa_mens_lacrosse_results_2024 (1).csv
User uploaded file "ncaa_mens_lacrosse_results_2014 (1

In [ ]:
elo_system = EloRatingSystem()


In [ ]:
import pandas as pd
import io

# Assuming 'uploaded' is the dictionary returned by files.upload() containing the content of the files
all_data = []
for filename, content in uploaded.items():
    df = pd.read_csv(io.BytesIO(content))  # Read each file from memory
    all_data.append(df)

# Concatenate all the DataFrames into one DataFrame
combined_data = pd.concat(all_data, ignore_index=True)
combined_data['Date'] = pd.to_datetime(combined_data['Date'])  # Convert 'Date' to datetime
combined_data.sort_values('Date', inplace=True)  # Sort the data by date

# Show the first few rows to confirm
print(combined_data.head())


         Date          Round Status      Team 1  Rank 1  Score 1     Team 2  \
0  2014-02-08  Unknown Round  Final      Denver     NaN     14.0  Air Force   
10 2014-02-08  Unknown Round    NaN  High Point     NaN      0.0     Towson   
8  2014-02-08  Unknown Round    NaN   Boston U.     NaN      0.0     Mercer   
7  2014-02-08  Unknown Round  Final         VMI     NaN      5.0       Navy   
6  2014-02-08  Unknown Round  Final    Delaware     NaN      4.0   Bucknell   

    Rank 2  Score 2    Winner  
0      NaN      8.0    Denver  
10     NaN      0.0       NaN  
8      NaN      0.0       NaN  
7      NaN     18.0      Navy  
6      NaN      6.0  Bucknell  


In [ ]:
# Update ratings based on the game results
for index, row in combined_data.iterrows():
    if pd.notna(row['Score 1']) and pd.notna(row['Score 2']):
        score1, score2 = int(row['Score 1']), int(row['Score 2'])
        elo_system.update_ratings(row['Team 1'], row['Team 2'], score1, score2)

# Extract and display the final ratings
final_ratings = pd.DataFrame(list(elo_system.get_ratings().items()), columns=['Team', 'Rating']).sort_values(by='Rating', ascending=False)
print(final_ratings)



            Team       Rating
55    Notre Dame  2502.348460
13      Maryland  2208.574371
35    Georgetown  2133.758061
19          Duke  2129.398427
53      Virginia  2113.333217
..           ...          ...
29      Canisius   875.231560
96    Lindenwood   854.579496
40        Wagner   738.896200
84  UMass Lowell   714.948503
70       Hampton   604.484256

[107 rows x 2 columns]


In [ ]:
def display_elo_ratings(elo_system):
    # Ensure the ratings are sorted by Elo rating value. This sorts from highest to lowest by default.
    sorted_ratings = sorted(elo_system.ratings.items(), key=lambda x: x[1], reverse=True)

    # Print the sorted ratings
    print("Team Names and Their Elo Ratings:")
    for team, rating in sorted_ratings:
        print(f"{team}: {rating:.2f}")


display_elo_ratings(elo_system)


Team Names and Their Elo Ratings:
Notre Dame: 2502.35
Maryland: 2208.57
Georgetown: 2133.76
Duke: 2129.40
Virginia: 2113.33
Denver: 2099.97
Johns Hopkins: 2078.04
Princeton: 2044.60
Penn St.: 2029.87
Syracuse: 2026.73
Michigan: 2001.05
Cornell: 1962.09
Army West Point: 1939.48
Saint Joseph's: 1927.98
Yale: 1926.08
Penn: 1913.59
North Carolina: 1882.88
Towson: 1847.10
Richmond: 1832.02
Loyola (Md.): 1827.88
Boston U.: 1815.27
Lehigh: 1771.04
Villanova: 1759.65
Utah: 1754.03
Rutgers: 1748.88
Harvard: 1734.00
Jacksonville: 1729.62
Ohio St.: 1698.92
Delaware: 1688.14
Loyola Maryland: 1677.75
Hartford: 1673.95
UAlbany: 1664.24
High Point: 1646.31
Albany (NY): 1639.39
Bryant: 1610.61
Army: 1607.10
Navy: 1584.65
Colgate: 1571.96
Providence: 1561.76
Air Force: 1557.54
Cleveland State: 1550.26
Massachusetts: 1543.89
Vermont: 1542.83
Brown: 1527.44
Tufts: 1526.35
Greensboro: 1500.30
Roberts Wesleyan: 1500.00
Thiel: 1500.00
Westminster (PA): 1500.00
Bethany (WV): 1500.00
Chowan: 1500.00
Cabrini: 

In [ ]:
def calculate_win_probability(elo_system, team1, team2):
    if team1 in elo_system.ratings and team2 in elo_system.ratings:
        rating1 = elo_system.ratings[team1]
        rating2 = elo_system.ratings[team2]
        expected_score_team1 = 1 / (1 + 10 ** ((rating2 - rating1) / 400))
        expected_score_team2 = 1 - expected_score_team1
        return expected_score_team1, expected_score_team2
    else:
        return None  # Handle case where one or both teams are not in the ratings




In [ ]:
team1 = input("Enter the name of Team 1: ")
team2 = input("Enter the name of Team 2: ")

probabilities = calculate_win_probability(elo_system, team1, team2)
if probabilities:
    print(f"Probability of {team1} winning: {probabilities[0] * 100:.2f}%")
    print(f"Probability of {team2} winning: {probabilities[1] * 100:.2f}%")
else:
    print("One or both of the teams are not found in the current Elo ratings.")


Enter the name of Team 1: Siena
Enter the name of Team 2: Dartmouth
Probability of Siena winning: 36.18%
Probability of Dartmouth winning: 63.82%
